In [2]:
import pandas as pd 
import sqlite3
import scipy.stats as sps
import matplotlib.pyplot as plt
from dataclasses import dataclass
import numpy as np

In [3]:
plt.style.use('ggplot')

In [4]:
with sqlite3.connect('../data_source/dota2base.db') as c:
    # general_table = pd.read_sql('select max(match_seq_num) from general_stats_table',con=c)
    general_stats_table = pd.read_sql('select * from general_stats_table',con=c)
    match_table = pd.read_sql('select * from match_table',con=c)

In [5]:
with sqlite3.connect('../data_source/dota2base.db') as c:
    hero_names_map =  pd.read_sql('select * from hero_names_map',con=c,index_col='index')['name'].to_dict()

In [6]:
match_table = match_table.apply(pd.to_numeric, downcast='unsigned')

In [7]:
filter_idx = match_table.groupby(['match_id','team_number'])['hero_id'].transform('nunique') == 5

In [10]:
pre_matrix = match_table.loc[filter_idx,['match_id','team_number','hero_id','radiant_win']]

In [12]:
pre_matrix['is_team_win'] = (1 - pre_matrix['team_number']) * pre_matrix['radiant_win'] + pre_matrix['team_number'] * (1 -pre_matrix['radiant_win'])

In [56]:
pre_matrix.to_hdf('data.h5',key='match_data',complevel=9)
pd.Series(hero_names_map).to_hdf('data.h5',key='names_map')
general_stats_table.to_hdf('data.h5',key='gen_stats',complevel=9)